In [9]:
import json
import pandas as pd

# Загружаем JSON
with open('data/public_transport_routes.json', encoding='utf-8') as f:
    data = json.load(f)

rows = []

# Обходим все маршруты
for line, directions in data['result'].items():
    for direction, stops in directions.items():
        for stop_number, stop_info in stops.items():
            row = {
                'line': line,
                'direction': direction,
                'stop_number': stop_number,
                **stop_info
            }
            rows.append(row)

df_bus_routes = pd.DataFrame(rows)

# Преобразуем stop_number в число
numeric_fields = ['stop_number', 'odleglosc']

for col in numeric_fields:
    df_bus_routes[col] = pd.to_numeric(df_bus_routes[col], errors='coerce')  # если вдруг где-то окажется пустота или ошибка — станет NaN

# Теперь сортировка будет корректной
df_bus_routes = df_bus_routes.sort_values(by=['line', 'direction', 'stop_number'])

df_bus_routes = df_bus_routes.rename(columns={
    'nr_zespolu': 'bus_stop_group',
    'odleglosc': 'distance',
    'nr_przystanku': 'bus_stop_platform'
})


df_bus_routes.to_parquet('data/public_transport_flat.parquet', index=False)



In [10]:
df_bus_routes.head()

,line,direction,stop_number,distance,ulica_id,bus_stop_group,typ,bus_stop_platform
47421,1,TD-1AN03,1,0,1501,R-01,6,00
47423,1,TD-1AN03,2,182,0123,5004,5,03
47422,1,TD-1AN03,3,717,1703,5003,1,08
47425,1,TD-1AN03,4,938,1703,5084,1,02
47424,1,TD-1AN03,5,1213,1703,5085,1,02


In [73]:
import csv
import pandas as pd

# Read and prepare the dataframe
with open('data/timetable.csv', encoding='utf8') as f:
    reader = csv.DictReader(f)
    timetable = list(reader)

df_timetable = pd.DataFrame(timetable)
df_timetable = df_timetable.rename(columns={'next_bus_stop': 'direction'})

# Convert types
df_timetable['bus_stop_group_int'] = df_timetable['bus_stop_group'].astype(int)
df_timetable['bus_stop_platform_int'] = df_timetable['bus_stop_platform'].astype(int)
df_timetable['time_dt'] = pd.to_datetime(df_timetable['time'], format='%H:%M:%S')

# Step 1: sort by time within group
df_timetable = df_timetable.sort_values(by=['route', 'direction', 'bus_stop_group_int', 'bus_stop_platform_int', 'time_dt'])

# Step 2: get the earliest time per group to sort the groups
group_min_time = (
    df_timetable.groupby(['route', 'direction', 'bus_stop_group_int', 'bus_stop_platform_int'])['time_dt']
    .min()
    .reset_index()
    .rename(columns={'time_dt': 'group_min_time'})
)

# Step 3: merge this back to original df
df_timetable = df_timetable.merge(group_min_time, on=['route', 'direction', 'bus_stop_group_int', 'bus_stop_platform_int'])

# Step 4: sort by route, then by earliest time of the group, then by time within group
df_timetable = df_timetable.sort_values(by=['route', 'direction', 'group_min_time', 'bus_stop_group_int', 'bus_stop_platform_int', 'time_dt'])


In [26]:
import json
import pandas as pd

def parse_coords_item(item):
    row_dict = {}
    for kv in item.get("values", []):
        key = kv.get("key")
        value = kv.get("value")
        if key is not None:
            row_dict[key] = value
    return row_dict

# Читаем файл
with open('data/coordinates-of-stops.json', encoding='utf8') as f:
    data = json.load(f)

# Берём только данные внутри ключа "result"
raw_items = data.get("result", [])

# Обрабатываем
bus_stops = [parse_coords_item(elem) for elem in raw_items]

# В DataFrame
df_bus_stops = pd.DataFrame(bus_stops)

df_bus_stops = df_bus_stops.rename(columns={
    'zespol': 'bus_stop_group',
    'slupek': 'bus_stop_platform',
    'nazwa_zespolu': 'bus_stop_name',
    'id_ulicy': 'street_id',
    'szer_geo': 'latitude',
    'dlug_geo': 'longitude',
    'kierunek': 'direction',
    'obowiazuje_od': 'valid_from'
})



In [53]:
print('df_bus_routes.columns.tolist()', df_bus_routes.columns.tolist())
print(df_bus_routes.head(2).to_string())

print('df_timetable.columns.tolist()', df_timetable.columns.tolist())
print(df_timetable.head(2).to_string())

print('df_bus_stops.columns.tolist()', df_bus_stops.columns.tolist())
print(df_bus_stops.head(2).to_string())


df_bus_routes.columns.tolist() ['line', 'direction', 'stop_number', 'distance', 'ulica_id', 'bus_stop_group', 'typ', 'bus_stop_platform', 'next_ulica_id', 'next_bus_stop_group', 'next_bus_stop_platform', 'destination_ulica_id', 'destination_bus_stop_group']
  line direction  stop_number  distance ulica_id bus_stop_group typ bus_stop_platform next_ulica_id next_bus_stop_group next_bus_stop_platform destination_ulica_id destination_bus_stop_group
0    1  TD-1AN03            1         0     1501           R-01   6                00          0123                5004                     03                 0104                       1087
1    1  TD-1AN03            2       182     0123           5004   5                03          1703                5003                     08                 0104                       1087
df_timetable.columns.tolist() ['route', 'bus_stop_group', 'bus_stop_platform', 'direction', 'time', 'bus_stop_group_int', 'bus_stop_platform_int', 'time_dt', 'group_min_

In [49]:

# For each group (same line and direction), shift the ulica_id, bus_stop_group, and bus_stop_platform columns by -1
df_bus_routes['next_ulica_id'] = df_bus_routes.groupby(['line', 'direction'])['ulica_id'].shift(-1)
df_bus_routes['next_bus_stop_group'] = df_bus_routes.groupby(['line', 'direction'])['bus_stop_group'].shift(-1)
df_bus_routes['next_bus_stop_platform'] = df_bus_routes.groupby(['line', 'direction'])['bus_stop_platform'].shift(-1)


# Step 2: Compute destination info (last stop in the group)
destination_info = (
    df_bus_routes
    .sort_values('stop_number')  # ensure ordering
    .groupby(['line', 'direction'], as_index=False)
    .last()[['line', 'direction', 'ulica_id', 'bus_stop_group']]
    .rename(columns={
        'ulica_id': 'destination_ulica_id',
        'bus_stop_group': 'destination_bus_stop_group'
    })
)

# Step 3: Merge destination info back into full table
df_bus_routes = df_bus_routes.merge(destination_info, on=['line', 'direction'], how='left')

# Step 4: Drop rows where the next stop data is missing (typically the last stop)
df_bus_routes_extended = df_bus_routes.dropna(subset=['next_ulica_id', 'next_bus_stop_group', 'next_bus_stop_platform'])

# Step 5: Reorder/select columns
df_bus_routes_extended = df_bus_routes_extended[[
    'line', 'direction',
    'ulica_id', 'bus_stop_group', 'bus_stop_platform',
    'next_ulica_id', 'next_bus_stop_group', 'next_bus_stop_platform',
    'destination_ulica_id', 'destination_bus_stop_group'
]]



In [48]:
unique_directions_per_route = df_timetable.groupby('route')['direction'].apply(lambda x: sorted(set(x)))


In [52]:
import json
import pandas as pd

with open('data/public_transport_dictionary.json', encoding='utf8') as f:
    data = json.load(f)

# Достаём словарь улиц
streets = data.get("result", {}).get("ulice", {})

# Преобразуем в DataFrame
df_street_dictionary = pd.DataFrame(list(streets.items()), columns=["id", "street"])


In [77]:

# Load dictionary JSON
with open('data/public_transport_dictionary.json', encoding='utf8') as f:
    data = json.load(f)

# Extract street dictionary from 'ulice'
street_dict = data.get("result", {}).get("ulice", {})

# Convert to DataFrame
df_street_dictionary = pd.DataFrame(list(street_dict.items()), columns=["street_id", "street"])
df_street_dictionary["street_id"] = df_street_dictionary["street_id"].astype(str)

# Make lowercase versions of the join columns
df_timetable["direction_lower"] = df_timetable["direction"].str.lower()
df_street_dictionary["street_lower"] = df_street_dictionary["street"].str.lower()

# Perform the LEFT JOIN using the lowercase columns
# df_timetable_merged = df_timetable.merge(
#     df_street_dictionary,
#     how="left",
#     left_on="direction_lower",
#     right_on="street_lower"
# )

# Clean up (optional)
# df_timetable_merged = df_timetable_merged.drop(columns=["street", "street_lower", "direction_lower"])


In [65]:
unmatched_list = df_timetable_merged[df_timetable_merged["street_id"].isnull()][["direction"]].drop_duplicates()


In [63]:
group_counts = df_bus_routes.groupby(["line", "bus_stop_group", "bus_stop_platform", 'direction']).agg(
    directions_count=pd.NamedAgg(column="direction", aggfunc="nunique")
).reset_index()



In [78]:
from rapidfuzz import process, fuzz

# Step 1: Extract unique direction_lower values
unique_directions = pd.DataFrame(df_timetable["direction_lower"].unique(), columns=["direction_lower"])

# Step 2: Prepare the list of choices from the street dictionary
choices = df_street_dictionary["street_lower"].tolist()

# Define the fuzzy matching function
def fuzzy_match(direction, choices, threshold=70):
    best_match = process.extractOne(direction, choices, scorer=fuzz.token_set_ratio)
    if best_match and best_match[1] >= threshold:
        return best_match[0]
    return None

# Step 3: Calculate the best match for each unique direction_lower
unique_directions["best_match"] = unique_directions["direction_lower"].apply(lambda x: fuzzy_match(x, choices))

# Step 4: Merge the fuzzy match results with df_street_dictionary to get the street_id
mapping_df = unique_directions.merge(
    df_street_dictionary,
    how="left",
    left_on="best_match",
    right_on="street_lower"
)[["direction_lower", "street_id"]]

mapping_df["street_id"] = mapping_df["street_id"].astype(str)

# Step 5: Merge the mapping back to df_timetable
df_timetable_merged = df_timetable.merge(mapping_df, how="left", on="direction_lower")


In [85]:
df_timetable_merged_2 = df_timetable_merged.merge(
    df_bus_routes_extended[["line", "bus_stop_group", "bus_stop_platform", "next_bus_stop_group", "destination_ulica_id"]],
    how="left",
    left_on=["route", "bus_stop_group", "bus_stop_platform", "street_id"],
    right_on=["line", "bus_stop_group", "bus_stop_platform", "destination_ulica_id"]
)


In [86]:
df_timetable_merged_2 = df_timetable_merged_2.dropna(subset=["next_bus_stop_group", "destination_ulica_id"])


In [83]:
print(df_timetable_merged_2.columns.tolist())
print(df_timetable_merged_2.head(2).to_string())







['route', 'bus_stop_group', 'bus_stop_platform', 'direction', 'time', 'bus_stop_group_int', 'bus_stop_platform_int', 'time_dt', 'group_min_time', 'direction_lower', 'street_id', 'line', 'next_bus_stop_platform', 'destination_ulica_id']
  route bus_stop_group bus_stop_platform direction      time  bus_stop_group_int  bus_stop_platform_int             time_dt      group_min_time direction_lower street_id line next_bus_stop_platform destination_ulica_id
0     1           4003                12   Annopol  04:27:00                4003                     12 1900-01-01 04:27:00 1900-01-01 04:27:00         annopol      0104    1                     01                 0104
1     1           4003                12   Annopol  04:27:00                4003                     12 1900-01-01 04:27:00 1900-01-01 04:27:00         annopol      0104    1                     01                 0104


In [88]:
import pandas as pd

# Assume df_timetable is your DataFrame with the columns as shown

# # Convert time_dt to a datetime type if it isn't already.
# df_timetable['time_dt'] = pd.to_datetime(df_timetable['time_dt'])

# Create the new field time_of_day by rounding down to the nearest hour
df_timetable_merged_2['time_of_day'] = df_timetable_merged_2['time_dt'].dt.floor('H').dt.strftime('%Y-%m-%dT%H:00:00.000Z')

# Now group by bus_stop_group, next_bus_stop_platform and time_of_day, and count the events
agg_df = df_timetable_merged_2.groupby(['bus_stop_group', 'next_bus_stop_group', 'time_of_day']).size().reset_index(name='count_of_events')


C:\Users\nkhrustalev\AppData\Local\Temp\ipykernel_29356\3290082254.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_timetable_merged_2['time_of_day'] = df_timetable_merged_2['time_dt'].dt.floor('H').dt.strftime('%Y-%m-%dT%H:00:00.000Z')
